In [139]:
import pandas as pd
import numpy as np

## Translation Dictionary

In [140]:
terms_data = ""
dictionary = {}

with open('polmat.ttl', encoding='utf-8') as f:
    for line in f:
        terms_data += line
        
terms_data = terms_data.split(' .\n')

for element in terms_data:
    for part in element.split(';\n'):
        part = part.strip()
        
        if 'skos:prefLabel' in part:
            data = ''.join(part.split('      ')[1:]).split('\"')
            
            temp_dictionary = {}
            
            for i in range(1, len(data), 2):
                text = data[i].strip()
                
                text = '/'.join(text.split(' / '))
                # text = ';'.join(text.split(','))
                
                lang = data[i+1][1:3].strip()
                
                temp_dictionary[lang] = text
            
            try:
                dictionary[temp_dictionary['de']] = temp_dictionary['en']
                
            except:
                print('Does not contain all 3 languages')
                
with open('dutch_english.csv', 'w', encoding='utf-8') as f:
    f.write('Dutch,English\n')
    
    for key in dictionary:
        f.write(key + ',' + dictionary[key])
        f.write('\n')

Does not contain all 3 languages


In [141]:
big_topic_dict = {}
mapping_big_topic_dict = {}
dictionary_appended = {}

for key in dictionary:
    if key.split()[0].replace('.', '', 1).isdigit():
        
        if '.' in key.split()[0]:
            mapping_big_topic_dict[float(key.split()[0])] = big_topic_dict[key.split()[0].split('.')[0]]
        
        else:
            big_topic_dict[key.split()[0]] = dictionary[key]
            
        new_key = ' '.join(key.split(' ')[1:])
        
        dictionary_appended[new_key] = ' '.join(dictionary[key].split(' ')[1:])
        
dictionary = {**dictionary, **dictionary_appended}

## Processing main data

In [142]:
df = pd.read_csv("policey-bern_DHIntro.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_17648\657891122.py:1: DtypeWarning: Columns (40,41,42,49,87,92,97,102,119,132,137,139,140,141,143,144,145,147,148,149,151,152,153,154,156,157,158,159,161,162,163,164,166,167,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("policey-bern_DHIntro.csv")


In [143]:
df['New'] = df['RelatesTo'].isnull()
df[['New', 'RelatesTo']]

,New,RelatesTo
0,True,NaN
1,True,NaN
2,True,NaN
3,True,NaN
4,True,NaN
...,...,...
4927,True,NaN
4928,True,NaN
4929,True,NaN
4930,False,Verordnung 15.10.1795


In [144]:
for i in range(1, 26):
    num = 'Matters_level2_num' + "{:02d}".format(i)
    topic = 'Matters_level2_' + "{:02d}".format(i)
    
    df['Matters_level01_num' + "{:02d}".format(i)] = df[num].map(mapping_big_topic_dict)
    df[topic + '_el'] = df[topic].map(dictionary)

In [145]:
df.to_csv('policey-bern_DHIntro_translated.csv')

## Exploding

In [146]:
# Might want to add archives status in the future

core_columns = [
    'Date',
    'Form',
    'New'
]

In [147]:
df_explode = pd.DataFrame()

for i in range(1, 26):
    num = 'Matters_level01_num' + "{:02d}".format(i)
    topic = 'Matters_level2_' + "{:02d}".format(i) + '_el'

    temp = df[core_columns + [num, topic]]
    temp.rename(
            columns={
            num: 'Matters_level01',
            topic: 'Matters_level2'
        }, inplace=True
    )
    temp = temp.dropna()
    df_explode = pd.concat([df_explode, temp])

df_explode

C:\Users\User\AppData\Local\Temp\ipykernel_17648\1287583295.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(
C:\Users\User\AppData\Local\Temp\ipykernel_17648\1287583295.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(
C:\Users\User\AppData\Local\Temp\ipykernel_17648\1287583295.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(
C:\Users\User\AppData\Local\Temp\ipykernel_17648\1287583295.py:8

,Date,Form,New,Matters_level01,Matters_level2
0,1528-12-01T00:00:00Z,Verordnung,True,4 Economic order. Work and professional regula...,Forestry and Land Use
1,1528-02-17T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
2,1528-05-07T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
3,1528-05-30T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
4,1528-07-01T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
...,...,...,...,...,...
3310,1716-06-04T00:00:00Z,Maimandat,False,3 Social Services. Healthcare. Education. Culture,Poor Relief
3969,1743-12-05T00:00:00Z,Chorgerichtssatzung,True,3 Social Services. Healthcare. Education. Culture,Healthcare
3310,1716-06-04T00:00:00Z,Maimandat,False,3 Social Services. Healthcare. Education. Culture,Poor Relief
3969,1743-12-05T00:00:00Z,Chorgerichtssatzung,True,3 Social Services. Healthcare. Education. Culture,Poor Relief


In [148]:
df_explode.sort_values(by='Date')

,Date,Form,New,Matters_level01,Matters_level2
1,1528-02-17T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
1,1528-02-17T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
1,1528-02-17T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
1,1528-02-17T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
1,1528-02-17T00:00:00Z,Ordnung,True,1 Social Order and Religion,Matters of Religion
...,...,...,...,...,...
4927,1797-09-28T00:00:00Z,Publikation,True,4 Economic order. Work and professional regula...,Trade and Services
4928,1797-11-29T00:00:00Z,Verordnung,True,1 Social Order and Religion,Abundance and Luxury
4929,1797-12-09T00:00:00Z,Publikation,True,2 Public Order and Safety,Censorship
4930,1798-01-04T00:00:00Z,Verordnung,False,4 Economic order. Work and professional regula...,Trade and Services


In [149]:
df_explode.to_csv('matters_visualisation.csv')